In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [2]:
df = pd.read_csv("../data/SmileData.csv",skiprows=2)

C:\Users\Joselit\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (2,32,36,53,58,59,77,83,89,93,105,127,141,144) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df.head()

,CallReportNum,ReportVersion,LinkedToCallReportNum,CallDateAndTimeStart,CallDateAndTimeEnd,CallLength,CallerNum,PhoneWorkerNum,WasRealCall,WasHangup,...,SMILE ON 60+ Screening - Treatment Urgency,SMILE ON 60+ Screening - Untreated Decay,SMILE ON 60+ Site Information - Enroller ID,SMILE ON 60+ Site Information - Enroller Location,SMILE ON 60+ Treatment Plan - Action: Upload After Picture on Finish Tab,SMILE ON 60+ Treatment Plan - Is the treatment plan completed?,SMILE ON 60+ Treatment Plan - Was a treatment plan developed?,"SMILE ON 60+ Treatment Plan - Was the enrollee able to be restored to function (can chew) and ""social six"" esthetics (top front six teeth are present and disease free)",SMILE ON 60+ Verification of Eligibility - Transportation Arranged,SMILE ON 60+ Verification of Eligibility - Verification of Eligibility
0,93115324,SMILE ON 60+ Registration,NaN,9/30/2021 16:56,9/30/2021 16:57,1,3638169,123747,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,93115188,SMILE ON 60+ Registration,NaN,9/30/2021 16:53,9/30/2021 16:53,0,4431194,123747,NaN,NaN,...,NaN,NaN,NaN,Referral Follow-Up,NaN,NaN,NaN,NaN,NaN,NaN
2,93115081,SMILE ON 60+ Clinical,NaN,9/30/2021 16:50,9/30/2021 16:51,1,2941341,161979,NaN,NaN,...,NaN,NaN,Christ Community Health Services- Broad Avenue,Provider Site,NaN,No,Yes,No,NaN,NaN
3,93115032,SMILE ON 60+ Clinical,NaN,9/30/2021 16:48,9/30/2021 16:49,1,3823933,161979,NaN,NaN,...,NaN,NaN,Christ Community Health Services- Broad Avenue,Provider Site,NaN,No,Yes,No,NaN,NaN
4,93114968,SMILE ON 60+ Clinical,NaN,9/30/2021 16:46,9/30/2021 16:48,2,3348256,161979,NaN,NaN,...,NaN,NaN,Christ Community Health Services- Broad Avenue,Provider Site,NaN,No,Yes,No,NaN,NaN


In [4]:
empty_call_df = ['LinkedToCallReportNum','WasRealCall','WasHangup','WasSexCall','WasWrongNumber','WasPrankCall','WasSilentCall']
#print(empty_call_df)

In [5]:
df.drop(empty_call_df, inplace=True, axis=1)

In [6]:
empty_loc_df = ['VolunteerComments', 'CallersFeedback', 'TextField2','TextField3','TextField4','TextField5','TextField6','TextField7','TextField8','TextField9','TextField10', 'IPAddress', 'VolIdentifier','GeoCode','GeoAssignment','CensusDivision','CensusTrack','CensusReportingArea','211Region','ThirdPartyName', 'ThirdPartyOrganization', 'ThirdPartyAddress','ThirdPartyCity','ThirdPartyCounty', 'ThirdPartyStateProvince','ThirdPartyPostalCode']
#print(empty_loc_df)

In [7]:
df.drop(empty_loc_df, inplace=True, axis=1)

In [8]:
misc_df = ['PhoneType','Feedback','Reviewed','FeedbackStatus','OrgNum', 'CountryName']

In [9]:
df.drop(misc_df, inplace=True, axis=1)

In [10]:
#df.head()

In [11]:
clean_df = df.dropna(axis=1, how='all')
#clean_df.head()

In [12]:
clean_df.head()

,CallReportNum,ReportVersion,CallDateAndTimeStart,CallDateAndTimeEnd,CallLength,CallerNum,PhoneWorkerNum,CityName,CountyName,StateProvince,...,SMILE ON 60+ Screening - Treatment Urgency,SMILE ON 60+ Screening - Untreated Decay,SMILE ON 60+ Site Information - Enroller ID,SMILE ON 60+ Site Information - Enroller Location,SMILE ON 60+ Treatment Plan - Action: Upload After Picture on Finish Tab,SMILE ON 60+ Treatment Plan - Is the treatment plan completed?,SMILE ON 60+ Treatment Plan - Was a treatment plan developed?,"SMILE ON 60+ Treatment Plan - Was the enrollee able to be restored to function (can chew) and ""social six"" esthetics (top front six teeth are present and disease free)",SMILE ON 60+ Verification of Eligibility - Transportation Arranged,SMILE ON 60+ Verification of Eligibility - Verification of Eligibility
0,93115324,SMILE ON 60+ Registration,9/30/2021 16:56,9/30/2021 16:57,1,3638169,123747,Smyrna,Rutherford,TN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,93115188,SMILE ON 60+ Registration,9/30/2021 16:53,9/30/2021 16:53,0,4431194,123747,Nashville,Davidson,TN,...,NaN,NaN,NaN,Referral Follow-Up,NaN,NaN,NaN,NaN,NaN,NaN
2,93115081,SMILE ON 60+ Clinical,9/30/2021 16:50,9/30/2021 16:51,1,2941341,161979,Memphis,Shelby,TN,...,NaN,NaN,Christ Community Health Services- Broad Avenue,Provider Site,NaN,No,Yes,No,NaN,NaN
3,93115032,SMILE ON 60+ Clinical,9/30/2021 16:48,9/30/2021 16:49,1,3823933,161979,Memphis,Shelby,TN,...,NaN,NaN,Christ Community Health Services- Broad Avenue,Provider Site,NaN,No,Yes,No,NaN,NaN
4,93114968,SMILE ON 60+ Clinical,9/30/2021 16:46,9/30/2021 16:48,2,3348256,161979,Memphis,Shelby,TN,...,NaN,NaN,Christ Community Health Services- Broad Avenue,Provider Site,NaN,No,Yes,No,NaN,NaN


#remove and replace name columns
smileon.columns = [x.replace('current', 'replacement') for x in smileon.columns]

In [13]:
# Set it to None to display all columns in the dataframe
pd.set_option('display.max_columns', None)
clean_df.head()

,CallReportNum,ReportVersion,CallDateAndTimeStart,CallDateAndTimeEnd,CallLength,CallerNum,PhoneWorkerNum,CityName,CountyName,StateProvince,PostalCode,Narrative,EnteredByWorkerNum,EnteredOn,FeedbackFromPhoneWorkerNum,FeedbackFromPhoneWorkerName,ReferralsMade,SMILE ON 60+ Demographic Information - # of People in Household,Birth Year,SMILE ON 60+ Demographic Information - Other Language,SMILE ON 60+ Notes (Optional) - Notes,SMILE ON 60+ Oral Care Encounter - 1351 - Quantity,SMILE ON 60+ Oral Care Encounter - 1354 - Quantity,SMILE ON 60+ Oral Care Encounter - 2140 - Quantity,SMILE ON 60+ Oral Care Encounter - 2150 - Quantity,SMILE ON 60+ Oral Care Encounter - 2160 - Quantity,SMILE ON 60+ Oral Care Encounter - 2161 - Quantity,SMILE ON 60+ Oral Care Encounter - 230 - Quantity,SMILE ON 60+ Oral Care Encounter - 2330 - Quantity,SMILE ON 60+ Oral Care Encounter - 2331 - Quantity,SMILE ON 60+ Oral Care Encounter - 2332 - Quantity,SMILE ON 60+ Oral Care Encounter - 2335 - Quantity,SMILE ON 60+ Oral Care Encounter - 2390 - Quantity,SMILE ON 60+ Oral Care Encounter - 2391 - Quantity,SMILE ON 60+ Oral Care Encounter - 2392 - Quantity,SMILE ON 60+ Oral Care Encounter - 2393 - Quantity,SMILE ON 60+ Oral Care Encounter - 2394 - Quantity,SMILE ON 60+ Oral Care Encounter - 2915 - Quantity,SMILE ON 60+ Oral Care Encounter - 2920 - Quantity,SMILE ON 60+ Oral Care Encounter - 2933 - Quantity,SMILE ON 60+ Oral Care Encounter - 2940 - Quantity,SMILE ON 60+ Oral Care Encounter - 2950 - Quantity,SMILE ON 60+ Oral Care Encounter - 2954 - Quantity,SMILE ON 60+ Oral Care Encounter - 2980 - Quantity,SMILE ON 60+ Oral Care Encounter - 3220 - Quantity,SMILE ON 60+ Oral Care Encounter - 4341 - Quantity,SMILE ON 60+ Oral Care Encounter - 4342 - Quantity,SMILE ON 60+ Oral Care Encounter - 6930 - Quantity,SMILE ON 60+ Oral Care Encounter - 6980 - Quantity,SMILE ON 60+ Oral Care Encounter - 7111 - Quantity,SMILE ON 60+ Oral Care Encounter - 7140 - Quantity,SMILE ON 60+ Oral Care Encounter - 7210 - Quantity,SMILE ON 60+ Oral Care Encounter - 7220 - Quantity,SMILE ON 60+ Oral Care Encounter - 7230 - Quantity,SMILE ON 60+ Oral Care Encounter - 7240 - Quantity,SMILE ON 60+ Oral Care Encounter - 7241 - Quantity,SMILE ON 60+ Oral Care Encounter - 7250 - Quantity,SMILE ON 60+ Oral Care Encounter - 7261 - Quantity,SMILE ON 60+ Oral Care Encounter - 7286 - Quantity,SMILE ON 60+ Oral Care Encounter - 7310 - Quantity,SMILE ON 60+ Oral Care Encounter - 7311 - Quantity,SMILE ON 60+ Oral Care Encounter - 7320 - Quantity,SMILE ON 60+ Oral Care Encounter - 7321 - Quantity,SMILE ON 60+ Oral Care Encounter - 7410 - Quantity,SMILE ON 60+ Oral Care Encounter - 7411 - Quantity,SMILE ON 60+ Oral Care Encounter - 7471 - Quantity,SMILE ON 60+ Oral Care Encounter - 7510 - Quantity,SMILE ON 60+ Oral Care Encounter - 7971 - Quantity,SMILE ON 60+ Oral Care Encounter - 7972 - Quantity,SMILE ON 60+ Oral Care Encounter - 9110 - Quantity,SMILE ON 60+ Oral Care Encounter - 9951 - Quantity,SMILE ON 60+ Oral Care Encounter - Lab Assist narrative,SMILE ON 60+ Screening - Last Screening Date,SMILE ON 60+ Verification of Eligibility - Verification Date,SMILE ON 60+ Base-line Oral and Conditions Questions - Are you limited in what you can eat?,SMILE ON 60+ Base-line Oral and Conditions Questions - Dental Clinic,SMILE ON 60+ Base-line Oral and Conditions Questions - Did the client commit to referral?,SMILE ON 60+ Base-line Oral and Conditions Questions - Did you help in these other areas?,SMILE ON 60+ Base-line Oral and Conditions Questions - Do you need help in these other areas?,SMILE ON 60+ Base-line Oral and Conditions Questions - Have you ever felt that the appearance of your mouth and teeth affected your quality of life?,SMILE ON 60+ Base-line Oral and Conditions Questions - How would you describe the condition of your mouth and teeth?,SMILE ON 60+ Base-line Oral and Conditions Questions - What is the next step for client encounter?,SMILE ON 60+ Base-line Oral and Conditions Questions - What is t